<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/valve_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" /></a>

In [ ]:
#@title Valve simulation in NeqSim#@markdown This notebook demonstrates how valves are simulated in NeqSim with emphasis on Cv and Cg handling.

In [ ]:
%%capture!pip install neqsim==2.5.35import neqsimfrom neqsim.thermo.thermoTools import *import matplotlib.pyplot as pltimport numpy as np%matplotlib inline

# Cv and Cg coefficientsIn NeqSim the valve unit operation may be specified by a flow coefficient. The most common is **Cv**, normally defined in US units. For gas flow it is often convenient to use the gas sizing coefficient **Cg**. The relation between them is roughly$$ C_g pprox 22.67 	imes C_v $$Setting either value in NeqSim will control the flow through the valve.

In [ ]:
from neqsim.thermo import fluidfrom neqsim.process import clearProcess, stream, runProcess, valvefluid1 = fluid('srk')fluid1.addComponent('methane', 1.0)fluid1.setTemperature(20.0, 'C')fluid1.setPressure(80.0, 'bara')fluid1.setTotalFlowRate(2.0, 'MSm3/day')clearProcess()stream1 = stream(fluid1)valve1 = valve(stream1)valve1.setCv(15.0)valve1.setPercentValveOpening(100.0)valve1.setOutletPressure(70.0, 'bara')runProcess()print('Outlet pressure:', valve1.getOutStream().getPressure('bara'), 'bara')print('Outlet temperature:', valve1.getOutStream().getTemperature('C'), 'C')

# Valve sizingTo size a valve for a required flow we can calculate the needed Cv (or Cg) from the flow and pressure drop. For an incompressible fluid the relation is$$ C_v = \frac{Q}{\sqrt{\Delta P / ho}} $$where $Q$ is the volumetric flow rate and $ho$ the fluid density.The same approach can be used with Cg by converting between Cv and Cg.

In [ ]:
# Compute required Cv for water flowfrom math import sqrtwater = fluid('srk')water.addComponent('water',1.0)water.setTemperature(25.0,'C')water.setPressure(5.0,'bara')water.setTotalFlowRate(10.0,'m3/hr')rho = water.getDensity('kg/m3')DeltaP = 1.0  # barQ = 10.0       # m3/hrCv_required = Q / sqrt(DeltaP / (rho/1000.0))print('Required Cv:', round(Cv_required,2))

# Controlling downstream pressure by valve openingThe percent valve opening can be adjusted to reach a desired downstream pressure. The snippet below uses a simple iterative approach to find the opening giving approximately 60 bar downstream pressure.

In [ ]:
target = 60.0valve1.setCv(15.0)open = 10.0for i in range(10):    valve1.setPercentValveOpening(open)    runProcess()    p_out = valve1.getOutStream().getPressure('bara')    open *= target/p_outprint('Final opening', round(open,1),'%')print('Resulting downstream pressure', p_out,'bara')